# Décorations
- Python: @dataclass, @staticmethod, @classmethod, @property, @lru_cache
- Pytest: @pytest.fixture, @pytest.mark
- Api rest avec Flask ou Fastapi: routage

Tutoriel: https://realpython.com/primer-on-python-decorators/

In [22]:
import functools
from datetime import datetime
from time import sleep

## Décoration @lru_cache
LRU = Least Recent Used

In [2]:
def fibo_imperative(n):
    a = 0
    b = 1
    if n == 1:
        return a
    elif n == 2:
        return b
    for i in range(n-2):
        a, b = b, a + b
    return b

In [7]:
n = 100
%timeit res1 = fibo_imperative(n)

12.6 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
def fibo_rec(n):
    if n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibo_rec(n-1) + fibo_rec(n-2)

In [17]:
%timeit fibo_rec(40)

1min 11s ± 5.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
@functools.lru_cache
def fibo_rec_opt(n):
    if n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibo_rec_opt(n-1) + fibo_rec_opt(n-2)

In [15]:
%timeit res3 = fibo_rec_opt(n)

196 ns ± 12 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Décorateur @chrono

In [52]:
def chrono(fun):
    @functools.wraps(fun)
    def chrono_wrapper(*args,**kwargs):
        t1 = datetime.now()
        res = fun(*args,**kwargs)
        t2 = datetime.now()
        delta = t2 - t1
        print("Chrono:", delta)
        return res
    # chrono_wrapper.__name__ = fun.__name__
    return chrono_wrapper

In [53]:
@chrono
def f():
    """sleepy function
    sleep during 3 seconds
    """
    sleep(3)

In [58]:
assert f() is None

Chrono: 0:00:03.001201


In [55]:
# code précédent équivalent au suivant
# f2 = chrono(f)
# f2()

In [56]:
f?

Signature: f()
Docstring:
sleepy function
sleep during 3 seconds
File:      c:\users\matthias\appdata\local\temp\ipykernel_11700\3526985709.py
Type:      function


In [57]:
help(f)

Help on function f in module __main__:

f()
    sleepy function
    sleep during 3 seconds



In [51]:
f.__name__, f.__module__

('f', '__main__')

### Chronomètre Fibo imperatif

In [59]:
fibo_imperative = chrono(fibo_imperative)

In [62]:
fibo_imperative(400)

Chrono: 0:00:00


108788617463475645289761992289049744844995705477812699099751202749393926359816304226

### Chronometre Fibo recursif

In [64]:
@chrono
def fibo_rec(n):
    def _fibo_rec(n):
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return _fibo_rec(n-1) + _fibo_rec(n-2)
    return _fibo_rec(n)

In [66]:
fibo_rec(40)

Chrono: 0:00:56.708090


63245986

### Chronometre Fibo recursif with LRU cache

In [68]:
@chrono
def fibo_rec(n):
    @functools.lru_cache
    def _fibo_rec(n):
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return _fibo_rec(n-1) + _fibo_rec(n-2)
    return _fibo_rec(n)

In [69]:
fibo_rec(40)

Chrono: 0:00:00


63245986

In [70]:
fibo_rec(400)

Chrono: 0:00:00.001997


108788617463475645289761992289049744844995705477812699099751202749393926359816304226

### Decorateur twice

In [74]:
def twice(fun):
    @functools.wraps(fun)
    def count_wrapper(*args,**kwargs):
        res = fun(*args,**kwargs)
        print("function called once: {}".format(res))
        res = fun(*args,**kwargs)
        print("function called twice: {}".format(res))
        return res
    return count_wrapper   

In [75]:
@twice
def fibo_rec(n):
    @functools.lru_cache
    def _fibo_rec(n):
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return _fibo_rec(n-1) + _fibo_rec(n-2)
    return _fibo_rec(n)

In [76]:
fibo_rec(10)

function called once: 34
function called twice: 34


34

In [81]:
@chrono
@twice
def fibo_rec(n):
    @functools.lru_cache
    def _fibo_rec(n):
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return _fibo_rec(n-1) + _fibo_rec(n-2)
    return _fibo_rec(n)

In [82]:
fibo_rec(400)

function called once: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226
function called twice: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226
Chrono: 0:00:00.002004


108788617463475645289761992289049744844995705477812699099751202749393926359816304226

In [77]:
@twice
@chrono
def fibo_rec(n):
    @functools.lru_cache
    def _fibo_rec(n):
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return _fibo_rec(n-1) + _fibo_rec(n-2)
    return _fibo_rec(n)

In [79]:
fibo_rec(400)

Chrono: 0:00:00.001043
function called once: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226
Chrono: 0:00:00.000950
function called twice: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226


108788617463475645289761992289049744844995705477812699099751202749393926359816304226

### Decorateur repeat(n)

In [83]:
def repeat(n):
    def repeat_decorator(fun):
        @functools.wraps(fun)
        def repeat_wrapper(*args,**kwargs):
            res = None
            for i in range(n):
                res = fun(*args,**kwargs)
                print(f"function called [{i}]: {res}")
            return res
        return repeat_wrapper
    return repeat_decorator

In [84]:
@repeat(4)
@chrono
def fibo_rec(n):
    @functools.lru_cache
    def _fibo_rec(n):
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return _fibo_rec(n-1) + _fibo_rec(n-2)
    return _fibo_rec(n)

In [85]:
fibo_rec(400)

Chrono: 0:00:00.005456
function called [0]: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226
Chrono: 0:00:00
function called [1]: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226
Chrono: 0:00:00.001063
function called [2]: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226
Chrono: 0:00:00
function called [3]: 108788617463475645289761992289049744844995705477812699099751202749393926359816304226


108788617463475645289761992289049744844995705477812699099751202749393926359816304226